# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию

Импортируем библиотеку `Pandas`:

In [1]:
import pandas as pd

Прочитаем файл `data.csv` из папки `/datasets` и сохраним его в переменной `df`:

In [2]:
df = pd.read_csv('/datasets/data.csv')

Выводим на экран первые десять строк таблицы:

In [3]:
display(df)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля


Получаем общую информацию о таблице:

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Описание таблицы**

В таблице 12 столбцов и 21525 строк. Тип данных в столбцах — `float`, `int`, `object`.

Описание данных:

* `children` — количество детей в семье;
* `days_employed` — трудовой стаж в днях;
* `dob_years` — возраст клиента в годах;
* `education` — образование клиента;
* `education_id` — идентификатор образования;
* `family_status` — семейное положение;
* `family_status_id` — идентификатор семейного положения;
* `gender` — пол клиента;
* `income_type` — тип занятости;
* `debt` — имел ли задолженности по возврату кредитов;
* `total_income` — доход в месяц;
* `purpose` — цель получения кредита.

Количество значений в столбцах различается. Значит, в данных есть пропущенные значения.

Одни и теже цели получения кредита описаны по разному.

В колонке с трудовым стажем имеются вещественные числа c отрицательными значениями.

**Вывод**

В каждой строке таблицы — данные о клиенте. Часть колонок содержит информацию о поле, возрасте, семейном положении, наличии детей и образовании. Остальные данные рассказывают о типе занятости, трудовому стажу и доходу. Также есть информация о целях получения кредита и имел ли клиент задолженности по возврату кредитов.

По предварительно оцененной информации можно утверждать, что, данных достаточно для проверки гипотез. Но: встречаются пропуски в данных (в столбцах `"days_employed"` и `"total_income"` имеются пропущенные значения); в столбце `"days_employed"` имеются вещественные числа c отрицательными значениями; столбец `"education"` содержит значения с разным регистром символов; одни и теже цели получения кредита описаны разными словами.

Поэтому, чтобы перейти к проверке гипотез, нужно устранить проблемы в данных.

## Шаг 2. Предобработка данных

### Обработка пропусков

Посчитаем, сколько в таблице пропущенных значений.

In [5]:
df.isna().sum() # подсчёт пропусков

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Как и ожидалось столбцы `"days_employed"` и `"total_income"` имеют пропуски в размере `2174` значений, это порядка `10%` от общего числа клиентов (поэтому удалить мы их не можем, будем обрабатывать).

В рамках данного проекта не будем учитывать возможную корреляцию заменяемых значений от каких-либо других показателей, поэтому заменим пропуски на средние (медианы) по данным столбцам:

Напишем функцию замены пропусков (можно обойтись и без функции, но захотелось потренироваться):

In [6]:
def replace_columns(list_columns):
    '''
    Функция для замены пропущенных значений в столбцах.
    На вход принимает список столбцов "list_columns", в которых необходимо произвести замену пропусков.
    Заменяет пропуски средним (медианой) значением по итерируемому столбцу.
    '''
    for column in list_columns: # проходим по списку столбцов
        df[column] = df[column].fillna(df[column].mean()) # заменяем пропущенные значения на среднее (медиану)

Создаем список столбцов, в которых необходимо произвести замену пропусков.

In [7]:
list_columns = ['days_employed', 'total_income'] # создание нового списка

Выполняем функцию замены пропусков:

In [8]:
replace_columns(list_columns) # вызов функции с аргументом "list_columns"

Проверяем остались ли пропущенные значения в столбцах:

In [9]:
df.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

Замена произошла упешно - пропущенных значений в столбцах не осталось

**Вывод**

Столбцы `"days_employed"` и `"total_income"` имели порядка `10%` пропусков от общего числа клиентов. Причем количество пропусков в столбцах равное, из чего можно предположить, что клиент никогда не работал (по крайней мере официально), т.е. не имеет трудовой книжки и "белой" зарплаты, либо клиент просто не заполнял эти данные, так как кредитор не требует от заемщика официального трудоустройства.

### Замена типа данных

В процессе предварительного анализа было определено, что два столбца (`"days_employed"` и `"total_income"`) имеют вещественный тип данных. Для удобства дальнейшего использования данных из этих столбцов переведем значения в целочисленный вид.

Создадим новый список, значениями которого будут столбцы, подлежащие изменению:

In [10]:
list_meaning = ['days_employed', 'total_income'] # создаем список с названиями столбцов

Далее переберем его элементы циклом `for` и для каждого столбца выполним замену типов:

In [11]:
for columns in list_meaning:
    df[columns] = df[columns].astype('int')

Проверим изменился ли тип данных столбцов `"days_employed"` и `"total_income"` на `'int'`

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


Да, как видим изменение типов произошло успешно.

**Вывод**

В данном случае для изменения типа данных был использован метод `astype()` так как значениями были вещественные числа, но если бы значениями оказались строковые типы данных, то пришлось бы использовать метод `to_numeric()`.

### Обработка дубликатов

In [13]:
print(df['children'].unique()) # выведем список уникальных значений

[ 1  0  3  2 -1  4 20  5]


In [14]:
def replace_wrong_values(columns, old_name, correct_name):
    '''
    Функция заменяет в столбце «columns» значение «old_name» на новое значение «correct_name».
    Обозначения:
    - «columns» название итерируемого столбца;
    - «old_name» - содержит неправильное значение;
    - «correct_name» – содержит правильное значение.
    '''
    for i in old_name:
        df[columns] = df[columns].replace(old_name, correct_name)

In [15]:
replace_wrong_values('children', [-1], 1) # вызов функции для замены «-1» на «1»
replace_wrong_values('children', [20], 2) # вызов функции для замены «20» на «2»

In [16]:
print(df['children'].unique())

[1 0 3 2 4 5]


In [17]:
print(df['education'].unique()) # выведем список уникальных значений

['высшее' 'среднее' 'Среднее' 'СРЕДНЕЕ' 'ВЫСШЕЕ' 'неоконченное высшее'
 'начальное' 'Высшее' 'НЕОКОНЧЕННОЕ ВЫСШЕЕ' 'Неоконченное высшее'
 'НАЧАЛЬНОЕ' 'Начальное' 'Ученая степень' 'УЧЕНАЯ СТЕПЕНЬ'
 'ученая степень']


In [18]:
df['education'] = df['education'].str.lower() #преобразует символы значения в нижний регистр методом 'str.lower()'

In [19]:
print(df['education'].unique())

['высшее' 'среднее' 'неоконченное высшее' 'начальное' 'ученая степень']


Посчитаем явные дубликаты в таблице:

In [20]:
df.duplicated().sum()
print("Количество явных дубликатов: {}".format(df.duplicated().sum()))

Количество явных дубликатов: 71


Выяснили, что таблица содержит 71 явный дубликат.

Сохраним дубликаты в новую таблицу и выведем первые пять строк на экран:

In [21]:
duplicated_df = df[df.duplicated()].head()
display(duplicated_df)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
2849,0,63046,41,среднее,1,женат / замужем,0,F,сотрудник,0,167422,покупка жилья для семьи
3290,0,63046,58,среднее,1,гражданский брак,1,F,пенсионер,0,167422,сыграть свадьбу
4182,1,63046,34,высшее,0,гражданский брак,1,F,сотрудник,0,167422,свадьба
4851,0,63046,60,среднее,1,гражданский брак,1,F,пенсионер,0,167422,свадьба
5557,0,63046,58,среднее,1,гражданский брак,1,F,пенсионер,0,167422,сыграть свадьбу


Удалим явные дубликаты методом `drop_duplicates()` (для удаления старых индексов и формирования новых применим метод `reset_index()`):

In [22]:
df = df.drop_duplicates()

Ещё раз посчитаем явные дубликаты в таблице — убедимся, что полностью от них избавились:

In [23]:
df.duplicated().sum() # проверка на отсутствие дубликатов
print("Количество явных дубликатов: {}".format(df.duplicated().sum()))

Количество явных дубликатов: 0


**Вывод**

При поиске явных дубликатов обнаружилось, что их количество составляет 71 значение. 

Для поиска использовали метод `"duplicated()"`, который возвращает `Series` со значением `True` именно для строк-дубликатов. Метод `value_counts()` и `unique` здесь не подходят, так как они анализируют определенный столбец, выбирают каждое уникальное значение и подсчитывают частоту его встречаемости.

Для удаления дубликатов использовался метод `drop_duplicated()` после применения которого нарушается индексация, и что бы ее обновить применяли метод `"reset_index()"` с параметром `"drop=True"`, который позволяет не создавать дополнительный столбец `"index"` после применения метода `"reset_index()"`.

Причинами возникновения дубликатов могут быть повторные представления, неправильное соединение данных из разных источников, ошибки пользователя при занесении информации.

### Лемматизация

Для лемматизации целей кредита приведем значения столбца `"purpose"` к их словарной форме. Для этого необходимо подключить библиотеку `"PyMystem"`.

In [24]:
# импортируем модуль pymystem3
from pymystem3 import Mystem
m = Mystem() # присваиваем ее переменной m

In [25]:
list = str(df['purpose'].tolist()) # создаем список значений столбца 'purpose' для лемматизации
lemmas = m.lemmatize(list) # применяем лемматизацию к списку

Подсчитаем число упоминаний лемматизированных слов в списке `"lemmas"`:

In [26]:
# Для подсчета вызовем специальный контейнер Counter из модуля collections:
from collections import Counter
print(Counter(lemmas)) # выводим на экран полученные значения

Counter({' ': 33570, "', '": 21453, 'недвижимость': 6351, 'покупка': 5897, 'жилье': 4460, 'автомобиль': 4306, 'образование': 4013, 'с': 2918, 'операция': 2604, 'свадьба': 2324, 'свой': 2230, 'на': 2222, 'строительство': 1878, 'высокий': 1374, 'получение': 1314, 'коммерческий': 1311, 'для': 1289, 'жилой': 1230, 'сделка': 941, 'дополнительный': 906, 'заниматься': 904, 'подержать': 853, 'проведение': 768, 'сыграть': 765, 'сдача': 651, 'семья': 638, 'собственный': 635, 'со': 627, 'ремонт': 607, 'приобретение': 461, 'профильный': 436, 'подержанный': 111, "['": 1, "']\n": 1})


**Вывод**

Из лемматизированных данных видно что основными целями кредита являются: недвижимость(жилье), автомобиль, образование и свадьба.

### Категоризация данных

**2.5.1** Для категоризации целей кредита напишем функцию:

In [27]:
def category(purpose):
    '''
    Функция принимает на вход значение столбца 'purpose'
    и заменяет его на нужное.
    Возвращает категорию целей кредита.
    '''
    if 'недвижимост' in purpose:
        return 'недвижимость'
    if 'жиль' in purpose:
        return 'недвижимость'
    if 'авто' in purpose:
        return 'автомобиль'
    if 'образован' in purpose:
        return 'образование'
    if 'свадьб' in purpose:
        return 'свадьба'

Создадим новый столбец `'purpose_group'` для категорий кредитов:

In [28]:
df['purpose_group'] = df['purpose'].apply(category) # используем метод "apply()" и в качестве аргумента выступит функция

Проверим список уникальных значений в новом столбце `'purpose_group'`:

In [29]:
print(df['purpose_group'].unique()) 

['недвижимость' 'автомобиль' 'образование' 'свадьба']


**Так же для дальнейшего анализа нам потребуется категоризировать еще 3 столбца:**

* `children` — количество детей в семье;
* `family_status` — семейное положение;
* `total_income` — доход в месяц.

**2.5.2** Для категоризации столбца `children` напишем функцию:

Для выделения категорий примем следующее:
* если детей у клиента нет то категория `"бездетные"`;
* если у клиента 1 или 2 ребенка - категория `"малодетные"`;
* если у клиента более 2 детей, то категория `"многодетные"`.

In [30]:
def children_status(children):
    '''
    Функция принимает на вход значение столбца 'children'
    и заменяет его на нужное.
    Возвращает категорию 'бездетные' если значение столбца 'children' == 0,
    или возвращает категорию 'малодетные' если значение столбца 1 <= 'children' <= 2,
    иначе возвращает 'многодетные'.
    '''
    if children == 0:
        return 'бездетные'
    if 1 <= children <= 2:
        return 'малодетные'
    return 'многодетные'

In [31]:
# используем метод "apply()" и в качестве аргумента выступит функция
df['children_status'] = df['children'].apply(children_status)

In [32]:
# Проверим список уникальных значений в новом столбце 'children_status':
print(df['children_status'].unique())

['малодетные' 'бездетные' 'многодетные']


**2.5.3** Для категоризации столбца `family_status` напишем похожую функцию, только значения будем использовать из стобца `family_status_id`:

In [33]:
def family_status_category(family_status_id):
    '''
    Функция принимает на вход значение столбца 'family_status_id'
    и заменяет его на нужное.
    Возвращает категорию 'женат/замужем' если значение столбца 'family_status_id' == 0 или 1
    иначе возвращает 'не женат/не замужем'.
    '''
    if family_status_id == 0:
        return 'женат/замужем'
    if family_status_id == 0:
        return 'женат/замужем'
    return 'не женат/не замужем'

In [34]:
# используем метод "apply()" и в качестве аргумента выступит функция
df['family_status_category'] = df['family_status_id'].apply(family_status_category)

In [35]:
# Проверим список уникальных значений в новом столбце 'family_status_category':
print(df['family_status_category'].unique())

['женат/замужем' 'не женат/не замужем']


**2.5.4** Для категоризации столбца `total_income` напишем похожую функцию:

Для удобства расчетов примем следующие категории:
* `низкая ЗП` для значений до 100 000;
* `средняя ЗП` для значений от 101 000 до 200 000;
* `высокая ЗП` для значений 201 000 и выше.

In [36]:
def total_income_category(total_income):
    '''
    Функция принимает на вход значение столбца 'total_income'
    и заменяет его на нужное.
    Возвращает категорию:
    - 'низкая ЗП' если значение столбца 'total_income' <= 100000;
    - 'средняя ЗП'если значение столбца 101000 <= 'total_income' <= 200000;
    - иначе возвращает 'высокая ЗП'.
    '''
    if total_income <= 100000:
        return 'низкая ЗП'
    if (total_income >= 101000) and (total_income <= 200000):
        return 'средняя ЗП'
    return 'высокая ЗП'

In [37]:
# используем метод "apply()" и в качестве аргумента выступит функция
df['total_income_category'] = df['total_income'].apply(total_income_category)

In [38]:
# Проверим список уникальных значений в новом столбце 'family_status_category':
print(df['total_income_category'].unique())

['высокая ЗП' 'средняя ЗП' 'низкая ЗП']


На всякий случай проверим отсутствие пропусков после замены и наличие новых столбцов с категориями:

In [39]:
print(df.isna().sum())

children                  0
days_employed             0
dob_years                 0
education                 0
education_id              0
family_status             0
family_status_id          0
gender                    0
income_type               0
debt                      0
total_income              0
purpose                   0
purpose_group             0
children_status           0
family_status_category    0
total_income_category     0
dtype: int64


Как видно все получилось!!!

Посмотрим как теперь выглядят первые 5 строк таблицы:

In [40]:
display(df.head())

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_group,children_status,family_status_category,total_income_category
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость,малодетные,женат/замужем,высокая ЗП
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль,малодетные,женат/замужем,средняя ЗП
2,0,-5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость,бездетные,женат/замужем,средняя ЗП
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование,многодетные,женат/замужем,высокая ЗП
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба,бездетные,не женат/не замужем,средняя ЗП


**Вывод**

Мы категоризировали 4 столбца с данными посредством написания функции и добавлением нового столбца со значением, которое эта функция возвращает. Для добавления новых столбцов Датафрейма использовали метод `apply()`. Такая категоризация поможет выделить нужные категории при проверке гипотез. Теперь можно переходить непосредственно к самой проверке гипотез.

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

**Для ответа на данный вопрос я попробую решить задачу двумя способами: через введение новых переменных, а затем проверю расчеты созданием сводной таблицы.**


**1 способ**

Для начала необходимо знать количество невозвратов кредита в срок для категорий `"дети есть"` и `"детей нет"`

Посчитаем количество клиентов с детьми и без. Для этого посчитаем сколько раз встречается каждая категория `"children_status"`:

In [41]:
# сохраним количество малодетных клиентов в переменной have_children
have_children = df[df['children_status'] == 'малодетные']['children_status'].value_counts().sum()
print("Количество малодетных клиентов: {}".format(have_children))

Количество малодетных клиентов: 6983


In [42]:
# сохраним количество многодетных клиентов в переменной have_more_children
have_more_children = df[df['children_status'] == 'многодетные']['children_status'].value_counts().sum()
print("Количество многодетных клиентов: {}".format(have_more_children))

Количество многодетных клиентов: 380


In [43]:
# сохраним количество бездетных клиентов в переменной have_no_children
have_no_children = df[df['children_status'] == 'бездетные']['children_status'].value_counts().sum()
print("Количество бездетных клиентов: {}".format(have_no_children))

Количество бездетных клиентов: 14091


Оказывается, что клиентов, у которых детей нет, в два раза больше, чем клиентов у которых дети есть.

In [44]:
# сохраним количество малодетных клиентов и невозврат кредита в срок в переменной debt_children
debt_children = df[df['children_status'] == 'малодетные']['debt'].sum()
print("Количество малодетных клиентов и невозврат кредита в срок: {}".format(debt_children))

Количество малодетных клиентов и невозврат кредита в срок: 647


In [45]:
# сохраним количество многодетных клиентов и невозврат кредита в срок в переменной debt_more_children
debt_more_children = df[df['children_status'] == 'многодетные']['debt'].sum()
print("Количество многодетных клиентов и невозврат кредита в срок: {}".format(debt_more_children))

Количество многодетных клиентов и невозврат кредита в срок: 31


In [46]:
# сохраним количество бездетных клиентов и невозврат кредита в срок в переменной debt_no_children
debt_no_children = df[df['children_status'] == 'бездетные']['debt'].sum()
print("Количество бездетных клиентов и невозврат кредита в срок: {}".format(debt_no_children))

Количество бездетных клиентов и невозврат кредита в срок: 1063


Посчитаем отношение малодетных клиентов и невозврат кредита в срок к общему количеству малодетных клиентов:

In [47]:
# результат сохраним в переменной debt_have_children
# дополнительно напишем обработчик исключений (при вызове ошибки деления на 0)
try:
    debt_have_children = debt_children / have_children
    print("Процент невозврата кредита в срок для малодетных клиентов равен: {:.1%}".format(debt_have_children))
except:
    print('Деление на 0, проверьте входные данные')

Процент невозврата кредита в срок для малодетных клиентов равен: 9.3%


Посчитаем отношение многодетных клиентов и невозврат кредита в срок к общему количеству многодетных клиентов:

In [48]:
# результат сохраним в переменной debt_have_more_children
# дополнительно напишем обработчик исключений (при вызове ошибки деления на 0)
try:
    debt_have_more_children = debt_more_children / have_more_children
    print("Процент невозврата кредита в срок для многодетных клиентов равен: {:.1%}".format(debt_have_more_children))
except:
    print('Деление на 0, проверьте входные данные')

Процент невозврата кредита в срок для многодетных клиентов равен: 8.2%


Посчитаем отношение клиентов, без детей и невозврат кредита в срок к общему количеству клиентов без детей:

In [49]:
# результат сохраним в переменной debt_no_have_children
# дополнительно напишем обработчик исключений (при вызове ошибки деления на 0)
try:
    debt_no_have_children = debt_no_children / have_no_children
    print("Процент невозврата кредита в срок для бездетных клиентов равен: {:.1%}".format(debt_no_have_children))
except:
    print('Деление на 0, проверьте входные данные')

Процент невозврата кредита в срок для бездетных клиентов равен: 7.5%


**2 способ**

Проверим результаты расчетов через создание сводной таблицы:

In [50]:
new_df = pd.pivot_table(df, index='children_status', values='debt') # создаем сводную таблицу с нужными столбцами
display(new_df) # выводим на экран

,debt
children_status,
бездетные,0.075438
малодетные,0.092654
многодетные,0.081579


Расчеты выполнены корректно!!!

**Вывод**

В процессе анализа зависимости между наличием детей и возвратом кредита в срок выяснилось, что процент невозврата кредита в срок для малодетных клиентов равен **`9.3%`**, процент невозврата кредита в срок для многодетных клиентов **`8.2%`**, а процент невозврата кредита в срок для бездетных клиентов **`7.5%`**. Отсюда следует вывод что, если у клиента есть дети, то процент невозврата кредита в срок **`выше на 0.7 - 1,1%`** по отношению к клиентам у которых детей нет.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

Для ответа на этот вопрос создадим сводную таблицу по заранее категоризированному столбцу `"family_status_category"`:

In [51]:
famyly_df = pd.pivot_table(df, index='family_status_category', values='debt') # создаем сводную таблицу с нужными столбцами
display(famyly_df) # выводим на экран

,debt
family_status_category,
женат/замужем,0.075452
не женат/не замужем,0.088865


**Вывод**

В процессе анализа зависимости между семейным положением и возвратом кредита в срок выяснилось, что процент невозврата кредита в срок для клиентов с семейным положением `не женат/не замужем` равна **`8.9%`**, а процент невозврата кредита в срок для клиентов с семейным положением `женат/замужем` **`7.5%`**. Отсюда следует вывод что, если клиент `состоит в браке`, то процент невозврата кредита в срок **`меньше на 1.4%`** по отношению к `не женатым/не замужным` клиентам.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Для ответа на этот вопрос создадим сводную таблицу по заранее категоризированному столбцу `"total_income_category"`:

In [52]:
income_df = pd.pivot_table(df, index='total_income_category', values='debt') # создаем сводную таблицу с нужными столбцами
display(income_df) # выводим на экран

,debt
total_income_category,
высокая ЗП,0.070314
низкая ЗП,0.079319
средняя ЗП,0.086610


**Вывод**

В процессе анализа зависимости между уровнем дохода и возвратом кредита в срок выяснилось, что процент невозврата кредита в срок для клиентов с `"низкой ЗП"` равен **`7.9%`**, процент невозврата кредита в срок для клиентов со `"средней ЗП"` **`8.6%`**, а процент невозврата кредита в срок для клиентов с `"высокой ЗП"` **`7.0%`**. Отсюда следует вывод что, у клиентов с `"высокой ЗП"` процент невозврата кредита в срок меньше по сравнению с остальными, "лидерами" в данной ситуации оказались клиенты со `"среней ЗП"`, у них процент невозврата кредита в срок выше чем у остальных (на **`0,7%`** больше по отношению к клиентам с `"низкой ЗП"` и на **`1,6%`** больше по отношению к клиентам с `"высокой ЗП"`).

- Как разные цели кредита влияют на его возврат в срок?

Для ответа на этот вопрос создадим сводную таблицу по заранее категоризированному столбцу `"purpose_group"`:

In [53]:
target_cred_df = pd.pivot_table(df, index='purpose_group', values='debt') # создаем сводную таблицу с нужными столбцами
display(target_cred_df) # выводим на экран

,debt
purpose_group,
автомобиль,0.093590
недвижимость,0.072334
образование,0.092200
свадьба,0.080034


**Вывод**

Самый высокий процент не возврата кредита в срок у клиентов, чья цель - `"автомобиль"` и `"образование"`, для клиентов которые берут кредит для `"недвижимости"` и `"свадьбы"` процент не возврата меньше на **`1,2%`** - **`2,2%`** по сравнению с предыдущими.

## Шаг 4. Общий вывод

**Из проведенного исследования стало ясно, что:**

- если у клиента `есть дети`, то процент невозврата кредита в срок выше на **`0.7 - 1,1%`** по отношению к бездетным клиентам;
- если клиент `состоит в браке`, то процент невозврата кредита в срок меньше на **`1.4%`** по отношению к "одиноким" клиентам;
- у клиентов с `"высокой ЗП"` процент невозврата кредита в срок меньше по сравнению с остальными, а у клиентов со `"средней ЗП"` процент невозврата кредита в срок выше чем у остальных (на **`0,7%`** больше по отношению к клиентам с `"низкой ЗП"` и на **`1,6%`** больше по отношению к клиентам с `"высокой ЗП"`);
- так же стало ясно, что самый высокий процент не возврата кредита в срок у клиентов, чья цель - `"автомобиль"` и `"образование"`, а для клиентов которые берут кредит для `"недвижимости"` и `"свадьбы"` процент не возврата меньше на **`1,2% - 2,2%`** по сравнению с предыдущими.